In [2]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser
from langchain.document_loaders import JSONLoader

########## 1. 도구를 정의 ##########

### 1-1. Search 도구 ###
search = TavilySearchResults(k=6)

### 1-2. JSON 문서 검색 도구 (Retriever) ###
# JSON 파일 경로
file_path = 'label_data.json'
# JSONLoader의 jq_schema 설정
jq_schema = """
.[] | {
    "filename": .filename,
    "original": .original,
    "id": .id,
    "date": .date,
    "conference_number": .conference_number,
    "question_number": .question_number,
    "meeting_name": .meeting_name,
    "generation_number": .generation_number,
    "committee_name": .committee_name,
    "meeting_number": .meeting_number,
    "session_number": .session_number,
    "agenda": .agenda,
    "law": .law,
    "qna_type": .qna_type,
    "context": .context,
    "context_learn": .context_learn,
    "context_summary": {
        "summary_q": .context_summary.summary_q,
        "summary_a": .context_summary.summary_a
    },
    "questioner": {
        "name": .questioner_name,
        "ID": .questioner_ID,
        "ISNI": .questioner_ISNI,
        "affiliation": .questioner_affiliation,
        "position": .questioner_position
    },
    "question": {
        "tag": .question.tag,
        "comment": .question.comment,
        "keyword": .question.keyword
    },
    "answerer": {
        "name": .answerer_name,
        "ID": .answerer_ID,
        "ISNI": .answerer_ISNI,
        "affiliation": .answerer_affiliation,
        "position": .answerer_position
    },
    "answer": {
        "tag": .answer.tag,
        "comment": .answer.comment,
        "keyword": .answer.keyword
    }
}
"""

# JSONLoader를 사용하여 파일 로드
loader = JSONLoader(file_path, jq_schema=jq_schema, text_content=False)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

# FAISS 인덱스를 저장할 파일 경로
index_path = 'faiss_index'

# 로컬 GPU에서 SentenceTransformer 모델 사용
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda')

# 문서로부터 임베딩 계산
embeddings = [embedding_model.encode(doc.page_content) for doc in split_docs]

# FAISS 인덱스를 생성하거나 기존 인덱스를 불러오기
if os.path.exists(index_path):
    print("저장된 인덱스를 불러옵니다.")
    vector = FAISS.load_local(index_path, embedding_model)
else:
    if embeddings:
        # split_docs에서 텍스트를 추출하여 임베딩과 함께 제공
        texts = [doc.page_content for doc in split_docs]
        
        # FAISS 인덱스 생성
        vector = FAISS.from_texts(texts, embeddings)
        vector.save_local(index_path)  # 생성한 인덱스를 저장
        print(f"FAISS 인덱스를 {index_path} 경로에 저장했습니다.")
    else:
        print("임베딩된 문서가 없습니다.")

# Retriever를 생성
if vector:
    retriever = vector.as_retriever()

    # retriever_tool 정의
    retriever_tool = create_retriever_tool(
        retriever,
        name="json_search",  # 도구의 이름을 입력
        description="use this tool to search information from the JSON document",
    )

    ### 1-3. tools 리스트에 도구 목록을 추가 ###
    tools = [search, retriever_tool]

    ########## 2. LLM 을 정의 ##########
    llm = ChatOpenAI(model="gpt-4o", temperature=0)

    ########## 3. Prompt 를 정의 ##########
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful assistant. "
                "Make sure to use the json_search tool for searching information from the JSON document. "
                "If you can't find the information from the JSON document, use the search tool for searching information from the web.",
            ),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ]
    )

    ########## 4. Agent 를 정의 ##########
    agent = create_tool_calling_agent(llm, tools, prompt)

    ########## 5. AgentExecutor 를 정의 ##########
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

    ########## 6. 채팅 기록을 수행하는 메모리를 추가 ##########
    store = {}

    def get_session_history(session_ids):
        if session_ids not in store:
            store[session_ids] = ChatMessageHistory()
        return store[session_ids]

    agent_with_chat_history = RunnableWithMessageHistory(
        agent_executor,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
    )

    ########## 7. Agent 파서를 정의 ##########
    agent_stream_parser = AgentStreamParser()

else:
    print("FAISS 인덱스를 생성하거나 불러오지 못했습니다.")


/home/elicer/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


AttributeError: 'list' object has no attribute 'embed_documents'

In [1]:
pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 3.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 17.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 7.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Agentic RAG
 : 질문에 따라 문서를 검색하여 답변(RAG)하거나, 인터넷 검색 도구(Tavily Search:)를 활용하여 답변하는 에이전트

**참고**

- **Agentic RAG**  : RAG 를 수행하되, Agent 를 활용하여 RAG 를 수행하는 에이전트

LangChain에서는 에이전트가 여러 툴을 조합하여 복잡한 문제를 해결하는 능력을 가지고 있기 때문에, Tool은 에이전트가 활용할 수 있는 기능적 요소
Agent 가 활용할 도구를 정의하여 Agent가 추론(reasoning)을 수행할 때 활용하도록 만들 수 있음

-  **도구(Tools)** :특정 작업(계산, 검색, API 호출 등)을 수행하는 도구
- **에이전트** : 사용자의 요청을 분석하고 적절한 툴을 사용하여 문제를 해결하는 주체


### 웹 검색도구: Tavily Search

- [Tavily Search API 발급받기](https://app.tavily.com/sign-in)

`.env`에 환경변수 등록

- `TAVILY_API_KEY=발급 받은 Tavily API KEY 입력`

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력0
logging.langsmith("Agents")

LangSmith 추적을 시작합니다.
[프로젝트명]
Agents


In [3]:
import os

os.environ['LANGCHAIN_PROJUECT'] = 'Agents'

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성
# k=6은 검색 결과를 6개까지 가져오겠다는 의미
search = TavilySearchResults(k=6)

`search.invoke` 함수는 주어진 문자열에 대한 검색을 실행

`invoke()` 함수에 검색하고 싶은 검색어를 넣어 검색을 수행


In [5]:
# 검색 결과
search.invoke("국회 회의록이란 무엇인가요?")

[{'url': 'https://m.blog.naver.com/kye486jong/221409661708',
  'content': '국회 국감 등에서 보면 ... 회의록이란? 회의의 진행과정, 내용, 결과 등을 기록하여 문서화한 것입니다. 속기사가 신속, 정확하게 회의록을 작성해서 회의의 투명성을 제고시키고, 신뢰성을 향상시키며, 분쟁 발생 시와 책임 소재를 명확히 할 수 있습니다.'},
 {'url': 'https://assembly.go.kr/portal/main/contents.do?menuNo=600179',
  'content': '회의록은 국회 또는 회의체의 회의 내용 및 결과 등을 사실대로 충실하게 기록한 문서이다. 회의록은 회의에 관한 공식 기록이며 회의에 관한 쟁점이 있을 때에는 유력한 증거가 되므로 회의경과에 대하여 사실대로 기록한다. 국회 회의록은 개의부터 산회까지 ...'},
 {'url': 'https://dataset.nanet.go.kr/',
  'content': '「국회회의록 빅데이터」는 국회회의록(본회의, 상임위원회 등)을 발언자, 키워드 등으로 검색하고 발언내용을 데이터셋으로 내려 받아 빅데이터 분석, 인공지능 학습 등에 이용할 수 있도록 하여 국회회의록을 다양한 형태로 공유･활용할 수 있는 서비스입니다.'},
 {'url': 'https://play.google.com/store/apps/details?id=kr.go.assembly.mrecord&hl=ko',
  'content': '국회회의록 앱은 제헌부터 현재까지 모든 회의록을 제공하는 어플리케이션입니다. 본회의, 상임위원회 및 국정감사 등 회의 내용을 확인할 수 있으며, 안건과 발언자 등을 이용하여 원하는 정보를 검색할 수 있습니다. 별도의 설치가 필요없는 이북뷰어는 ...'},
 {'url': 'https://tldv.io/ko/blog/meeting-notes-vs-minutes/',
  'content': '회의록이란 무엇인가요? 회의록은 회의 노트와 달리 회

### 문서 기반 문서 검색 도구: Retriever

내가 넣은 문서에 대해 조회를 수행할 retriever도 생성

**실습에 활용한 문서**

초보 투자자를 위한 증권과 투자 따라잡기.pdf

In [9]:
import json

# JSON 파일 로드
with open('label_data.json', 'r', encoding='utf-8') as f:
    label_data_json = json.load(f)

# 첫 번째 항목의 구조 확인
print(label_data_json[0])


{'filename': 'SRC_16대_2000_2000년10월20일_국정감사_교육위원회_0001(030043).xlsx', 'original': 'http://likms.assembly.go.kr/record/mhs-10-040-0040.do?conferNum=030043&fileType=PDF', 'id': '030043', 'date': '2000年10月20日(金)', 'conference_number': '030043', 'question_number': '0001', 'meeting_name': '국정감사', 'generation_number': '16', 'committee_name': '교육위원회', 'meeting_number': '2000', 'session_number': '2000년10월20일', 'agenda': '감사개시', 'law': '', 'qna_type': '추출형', 'context': '任鍾晳 위원입니다.  자료준비와 국정감사를 직접 받으시느라고 많이 피로하실 줄 압니다. 심심한 위로와 감사의 인사를 함께 드립니다.   저는 시간내에 일문일답으로 하겠습니다. 혹시 부족한 것이 있으면 서면으로 하겠습니다.   먼저 인천교육청에 질의 드리겠습니다.   우리가 씨랜드에서도 볼 수 있듯이 학생안전권은 학습권이나 복지권보다 훨씬 중요하다고 생각합니다. 앞서 金貞淑 위원님께서 말씀하셨지만 학생들의 유치원, 초ㆍ중ㆍ고등학교의 통학버스가 종합보험 특별약관에 미가입된 차량들이 전국에 18.2%나 됩니다.   인천도 27.3%가 종합보험 특별약관에 미가입되어서 불의의 사고시 굉장한 피해가 염려되고 있습니다. 서울과 대구, 광주, 경기도는 미가입문제가 깨끗이 정리되어 있고 나머지 지역은 좀 많은 데도 있고 적은 데도 있습니다. 인천은 전국 평균보다 약 10% 높은데 이것을 즉시 해결하는데 어떤 어려움이 있습니까? 말씀드리겠습니다.   인천시내에 초등학교 버스 미가입은 공립의 경우는 없고 사립은 거의 해결됐고 대부분이 유치원입니다. 사립유치원의 

In [11]:
import json
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

# JSON 파일 로드
with open('label_data.json', 'r', encoding='utf-8') as f:
    label_data_json = json.load(f)

# JSON 데이터를 Document 객체로 변환
# context 필드를 page_content로 사용하고, 다른 메타데이터는 그대로 유지
split_docs = [
    Document(
        page_content=doc['context'], 
        metadata={
            'filename': doc['filename'],
            'original': doc['original'],
            'date': doc['date'],
            'meeting_name': doc['meeting_name'],
            'committee_name': doc['committee_name'],
            'questioner_name': doc['questioner_name'],
            'answerer_name': doc['answerer_name']
        }
    ) 
    for doc in label_data_json
]

chunk_size = 100  # 한 번에 처리할 문서 수
for i in range(0, len(split_docs), chunk_size):
    # 현재 chunk의 문서들에 대해 벡터 생성
    chunk_docs = split_docs[i:i+chunk_size]
    vector = FAISS.from_documents(chunk_docs, OpenAIEmbeddings())

# Retriever 생성
retriever = vector.as_retriever()


 `retriever` 객체의 `invoke()` 를 사용하여 사용자의 질문에 대한 가장 **관련성 높은 문서** 를 찾는 데 사용


In [12]:
# 문서에서 관련성 높은 문서를 가져옴
retriever.invoke("가장 길었던 국회 회의 내용은 뭔지 요약적으로 알려줘")

[Document(metadata={'filename': 'SRC_21대_제400회_제3차_특별위원회_정치개혁특별위원회_0001(052127).xlsx', 'original': 'http://likms.assembly.go.kr/record/mhs-10-040-0040.do?conferNum=052127&fileType=PDF', 'date': '2022년9월15일(목)', 'meeting_name': '특별위원회', 'committee_name': '정치개혁특별위원회', 'questioner_name': '김영배', 'answerer_name': '임기근'}, page_content='제가 시간이 짧아서 좀 이따 한꺼번에 답변해 주시면 좋겠습니다.  첫 번째 질문이 이것을 일점일획도 못 바꾼다 이런 게 아니라면 심의․확정한다는 것을 정부가 제출한 예산안의 경미한 변경 이런 개념을 도입해서 한 2% 정도 내에서는 의회가 증액을 할 수 있도록 허용하는 이런 정도의 법률에 대해서는 어떻게 생각하시는지, 그게 헌법이 규정하고 있는 내용을 어기는 거냐, 어떻게 보시느냐 이걸 한번 여쭤보고 싶고요.  두 번째는, 두 번째 질문인데요. 아까 국정감사 시기를 말씀하신 분이 계셔서 이것은 전적으로 동의합니다. 현재 법률상으로도 국정감사와 조사에 관한 법률에 보면 국정감사는 정기회 전에 하도록 규정하고 있고 다만 국회가 따로 의결하면 정기회와 같이할 수 있다 이렇게 돼 있는데 ‘다만’이라고 하는 조항이 지금 국회 관행으로 굳어져 있습니다. 그래서 국정감사를 6월 전에 하는 것, 이게 현재 국정감사와 조사에 관한 법률의 원취지와도 부합하는데 국회가 그걸 어기고 있습니다. 그걸 지키도록 우리 위원회에서 의결하고 양 정당을 포함한 정당에다가, 교섭단체에 권고하는 게 어떤가 싶거든요. 이건 동료 위원님들도 의견을 주시면 좋겠습니다.  세 번째는 아까 톱다운 말씀을 주셨는데, 제가 교수님 성함을 모르겠는데 이것 보고, 제가 급해서 그렇습니다, 시간이 없어서. 톱다운 예산 제도를 도입했

이제 우리가 검색을 수행할 인덱스를 채웠으므로, 이를 에이전트가 제대로 사용할 수 있는 도구로 쉽게 변환 가능


`create_retriever_tool` 함수로 `retriever` 를 도구로 변환

In [13]:
from langchain.tools.retriever import create_retriever_tool


retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야함!!
)

### Agent 가 사용할 도구 목록 정의

Agent 가 사용할 도구 목록:  
  
`tools` 리스트는 `search`와 `retriever_tool`을 포함

In [14]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

#### Agent 생성

Agent 가 활용할 LLM을 정의하고, Agent 가 참고할 Prompt 를 정의

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

Tool Calling Agent 를 생성

In [18]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

마지막으로, 생성한 `agent` 를 실행하는 `AgentExecutor` 를 생성
- (참고)`verbose=False` 로 설정하여 중간 단계 출력을 생략

In [19]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

#### 에이전트 실행하기

현재 이러한 모든 질의는 **상태(Stateless) 가 없는** 질의 == 이전 상호작용을 기억하지 못함


`agent_executor` 객체의 `invoke` 메소드는 딕셔너리 형태의 인자를 받아 처리

In [20]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

In [22]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "가장 길었던 국회 회의록의 내용을 요약적으로 알려주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 가장 길었던 국회 회의록
Log: 
Invoking: `tavily_search_results_json` with `{'query': '가장 길었던 국회 회의록'}`



[관찰 내용]
Observation: [{'url': 'https://archives.nanet.go.kr/bbs/bbsView.do?bbs_id=2&bbs_no=48&curr_menu_cd=0102010000', 'content': '국회회의록; 국회의안 ... 춥고도 길었던 겨울, 그래도 그리운 그 시절 ... 연중 가장 춥다는 대한(大寒, 1.20.)을 앞두고, 1950년대~2000년대 그 시절의 겨울 풍경을 기록으로 만나본다. 행정자치부 국가기록원(원장 이상진)은 대한을 맞아 1월 이달의 기록 ...'}, {'url': 'https://scienceon.kisti.re.kr/srch/selectPORSrchArticle.do?cn=DIKO0015057924', 'content': '대한민국국회는 1948년 제헌국회 이후 70년의 역사를 이어오고 있다. 제헌 70년의 역사는 대한민국 현대사를 보여주는 거울이고, 제헌 70년의 역사를 가장 잘 기록하고 있는 것은 국회 회의록이다. 국회 회의록은 국회 회의의 내용 및 결과 등을 사실대로 충실하게 기록한 문서이다. 국회의원의 입법 ...'}, {'url': 'https://ko.wikisource.org/wiki/대한민국_제헌국회_제1회_제8차_본회의록', 'content': '1948년 6월 11일 오전 10시 10분, 대한민국 서울특별시 종로구 국회의사당. 제1회－제8호. 第 1回 國會速記錄. 국회본회의회의록. 第 8 號. 國 會 事 務 處. 4281년6월11일 (금) 상오 10시10분. 1948년6월11일 (금) 상오 10시10분. 국회 제8차 회의절차.'}, {'url': 'https://ko.wikisource.org/wiki/대한민국_제헌국회_제1

`agent_executor` 객체의 `invoke` 메소드를 사용하여, 질문을 입력으로 제공


In [23]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "안철수 의원이 발언한 날들을 찾아주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 안철수 의원 발언
Log: 
Invoking: `pdf_search` with `{'query': '안철수 의원 발언'}`



[관찰 내용]
Observation: 성일종 위원입니다.   후보자님, 자료 요청을 계속했는데 안 주시는 자료가 있어요. 오전에 전해 주신 따님의 자료는 사실 학교에서 줄 수 있는 자료를 미루다 미루다 늦게 주신 겁니다. 그리고 그 자료를 분석을 해 봤을 때 학비가 면제된 것이 사실이고 학교에서 일부를 받았지만 그 돈으로는 사실 미국에서 생활하기가 어렵습니다. 그렇지만 또 우리 후보자님께서 ‘제가 그 현장을 가 보지 않은 사람이, 거기 물가도 모르는 사람이 사실 대답하기가 어려움이 있지요. 그리고 또 할머니가 손주들을 사랑하고 그래서 주셨다’라고 하는 그 말의 대응논리가 그렇게 많지 않은 것도 사실입니다.   하지만 그것으로써 이게 다 해소가 됐다고 하기에는 한계가 있습니다. 자녀의 자산 증식에 대한 여러 가지 규모라든가 또 미국에서 써야 될 비용 그리고 오고가는 항공료, 여러 가지 다 계산해 보면 실질적으로 많은 돈이 갔다고 하는 것은 추론해 볼 수 있는 사항입니다. 이것을 분명히 아셨으면 좋겠고요.   두 번째, 후원회를 이렇게 많이 해 주셨는데 농협으로부터 농협 구좌로 있는 강성용 후원회장으로 후원해 준 것은 아주 다 정확하게 제가 요구하는 대로 일치가 다 됐습니다. 그러나 우리 후보자님께서 4개 구좌를 통해 가지고 후원을 했던 것들은 자료가 안 왔습니다. 이것은 좀 문제가 있다고 생각을 합니다.   그래서 이 부분에 대한, 지금 야당에서 이거 요구한다고 해서 이제 내주시겠습니까? 저는 불가능하다고, 안 내주실 거라고 보고요. 하여튼 이 부분에 대해서 상당히, 안 내주신 것에 대해서는 좀 유감스럽게 생각을 합니다. 제가 알고 있기로는 농협에서 일괄적으로 후원을 한 것으로 알고 있는데요. 자, 후보자님 이것은요 제가 이따가 그러면 의사진행발언을 얻어 가지고 하겠습니다. 질의

## 이전 대화내용 기억하는 Agent

`RunnableWithMessageHistory`: 이전의 대화내용을 기억하기 위해 사용  

`AgentExecutor`를 `RunnableWithMessageHistory`로 감싸줌

In [24]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [32]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "그럼 웹으로 검색해봐"},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2005년 12월 회의록 파일명
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2005년 12월 회의록 파일명'}`



[관찰 내용]
Observation: [{'url': 'https://www.mozilla.or.kr/blog/659', 'content': 'No responses yet. 2005년 12월 5일 열린 모질라 재단 임원 회의의 회의록 이 공개되었다. 파이어폭스 서밋 (Firefox Summit)과, 기술적인 부분에 대한 논의가 이루어졌다. 원문: MozillaZine: Minutes of the mozilla.org Staff Meeting of Monday 5th December 2005. 시몽키 1.0 베타 출시2005년 12월 20일.'}, {'url': 'https://www.mensakorea.org/bbs/board.php?bo_table=notice&wr_id=73', 'content': '2005년 총회 회의록: 등록일 l 05-02-06 13:14 조회 l 11447: 일시: 2005년 2월 5일. 토요일. 오후 2시~4시 장소: ... 출판분과 - 2, 5, 8, 12월 회지 발간 및 비용 발표 현 분과장 장현정님, 부분과장 최세영님 테스트분과 - 작년 ...'}, {'url': 'https://brunch.co.kr/@dreammentor/34', 'content': '여러 종류의 파일을 하나의 폴더에 보관해 놓아도 파일명 기준으로 자동 정렬이 되므로 관리하기도 편하다. 예를 들어 회의록 파일의 경우, "회의록_[업무명]_버전.ext"의 형식으로 파일명을 만들면, "회의록"이 분류 표시 역할을 하므로 파일을 분류해서 저장하기 ...'}, {'url': 'https://dataset.nanet.go.kr/', 'content': '「국회회의록 빅데이터」는 국회회의록(본회의,

In [22]:
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
Here is the translation of the previous answer into English:

1. **Nature of Capital**:
   - **Stocks**: Equity capital, representing ownership in a company.
   - **Futures**: A derivative product, a contract to buy or sell a specific asset at a predetermined price at a future date.

2. **Issuer**:
   - **Stocks**: Issued by corporations.
   - **Futures**: Traded as standardized contracts on exchanges.

3. **Position of the Owner**:
   - **Stocks**: Shareholders are owners of the company and can participate in management.
   - **Futures**: The owner of a futures contract does not own the asset; they only hold the rights under the contract.

4. **Duration**:
   - **Stocks**: Permanent.
   - **Futures**: Have an expiration date, and the contract ends when it expires.

5. **Profit Type and Nature**:
   - **Stocks**: Profits are distributed in the form of dividends, and the profits can be variable.
   - **Futures**: Profits and losses arise from the execution of the contract, and p

## Agent 템플릿

In [26]:
# PyMuPDF : 텍스트, 이미지, 주석, 레이아웃 등 PDF의 모든 요소 처리 가능한 패키지
! pip install PyMuPDF

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 50.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [33]:
pip install transformers sentence-transformers faiss-gpu


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 21.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 19.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 3.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 15.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 13.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 10.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB

In [34]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser
from langchain_core.documents import Document
import json

########## 1. 도구를 정의 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성
# k=6은 검색 결과를 6개까지 가져오겠다는 의미
search = TavilySearchResults(k=6)

### 1-2. JSON 파일 문서 검색 도구 (Retriever) ###
# JSON 파일 로드
with open("label_data.json", "r", encoding="utf-8") as f:
    label_data_json = json.load(f)

# JSON 데이터를 Document 객체로 변환
documents = [Document(page_content=doc['context'], metadata={"filename": doc["filename"]}) for doc in label_data_json]

# 텍스트 분할기를 사용하여 문서를 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = text_splitter.split_documents(documents)

# VectorStore를 생성
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성
retriever = vector.as_retriever()

# JSON 파일 검색용 도구 생성
retriever_tool = create_retriever_tool(
    retriever,
    name="json_search",  # 도구의 이름을 입력
    description="use this tool to search information from the JSON document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### 1-3. tools 리스트에 도구 목록을 추가 ###
# tools 리스트에 search와 retriever_tool을 추가
tools = [search, retriever_tool]

########## 2. LLM 을 정의 ##########
# LLM 모델을 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 를 정의 ##########

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `json_search` tool for searching information from the JSON document. "
            "If you can't find the information from the JSON document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의 ##########

# 에이전트를 생성
# llm, tools, prompt를 인자로 사용
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리를 추가 ##########

# session_id 를 저장할 딕셔너리 생성
store = {}

# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환

# 채팅 메시지 기록이 추가된 에이전트를 생성
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의합니다. ##########
agent_stream_parser = AgentStreamParser()


ImportError: cannot import name 'create_retriever_tool' from 'langchain.tools' (/home/elicer/.local/lib/python3.10/site-packages/langchain/tools/__init__.py)

In [30]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input":  "2005년 12월에 있었던 회의록 제목들을 알려줘."},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: json_search
query: 2005년 12월 회의록 제목
Log: 
Invoking: `json_search` with `{'query': '2005년 12월 회의록 제목'}`



[관찰 내용]
Observation: 의한 계열회사”하고 “공직자윤리법 제4조제1항의 규정에 의한 신고대상으로서 소유명의에 불구하고 지방자치단체의 장 또는 지방의회 의원이 사실상 소유하는 재산이 자본금 총액의 100분의 50 이상인 사업자”입니다. 이 정도로 하고 수의계약만으로 하면 괜찮을 것 같은데요. 입찰에는 여전히 참여할 자격이 있으니까 이렇게 되면 위헌의 소지는 많이 줄어들 것 같아요. 그러면 수정의견대로 하는데 이의 없습니까? 예. 부칙의 시행일이 지방계약법안은 2005년 7월 1일인데 2006년 1월 1일부터 시행하는 것으로 되겠습니다. 그것은 원래 저희가 12월에 제출하면서 그렇게 됐던 것이니까요. 그러면 수석전문위원이 다시 종합해서 간단하게 보고해 주시지요. 수정의견에 제시된 부분에 대해서는 정부 측과 충분히 협의했고 위원님들 대체토론에서 논의된 내용을 중심으로 반영한 내용입니다. 의사일정 제10항은 전문위원이 보고한 대로 수정한 부분은 수정한 대로, 나머지 부분은 원안대로 의결하고자 하는데 이의 없으십니까?     (「없습니다」 하는 위원 있음)  가결되었음을 선포합니다.  산회를 선언합니다.

하여튼 공공기관운영위원회 회의만큼은 회의록이 알리오 시스템에 바로 올라가거든요. 그래서 잘 이루어지고 있습니다.  그런데 그걸 다시 한번 강조하는 의미에서 법으로 규정을 올리는 문제이거든요. 제가 한전 사외이사를 한 적이 있는데요. 사외이사를 해 보면 사외이사가 이사회 의장을 합니다. 그리고 회의록이 다 공개되고 또 회의에서 발언한 내용이 혹시 틀렸는가 검토도 받고 하는데, 한전은 사외이사들이 오히려 경영진 야단도 많이 치고 굉장히 큰소리를 내요. 시장형 공기업을 그렇게 만들어 놨습니다. 그렇게 돼서 저는 한전 같은 경우는 공개라든지 이런 게

In [26]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요"},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
The history of the domestic securities market has undergone several changes. On March 27, 2000, the third market was established, and on July 13, 2005, the third market was relaunched as the Free Board. Later, on June 17, 2014, it was renamed K-OTC (Korea Over-The-Counter).

In 2005, recognizing the limitations of continuous growth and development due to the overlapping market operation system, and to prepare for the trend of exchange integration and international competition between markets, changes were made in the Korean securities market.

In the 1970s, construction stocks were prominent; in the 1980s, stocks related to high-tech industries such as electronics and automobiles gained attention. In the 1990s, with the allowance of direct foreign investment, industry-leading stocks emerged due to foreign investors. In the 2000s, with the development of the IT industry, internet and telecommunications-related stocks led the market. Around 2020, with the advent of the Fourth Ind

In [27]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {
        "input": "넷플릭스 TV 프로그램 흑백요리사 최종 우승자를 알려주세요. 한글로 답변하세요"
    },
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 넷플릭스 흑백요리사 최종 우승자
Log: 
Invoking: `tavily_search_results_json` with `{'query': '넷플릭스 흑백요리사 최종 우승자'}`



[관찰 내용]
Observation: [{'url': 'http://blog.naver.com/nyria99/223612999778?fromRss=true&trackingCode=rss', 'content': "5 days ago · 흑수저 '나폴리 맛피아(권성준)'가 결승전에서 백수저 '에드워드 리'를 상대로 승리하고 최종 우승을 하게 되었습니다. ... 넷플릭스 흑백요리사 - 백수저\xa0..."}, {'url': 'https://m.news.nate.com/view/20241009n09727', 'content': "6 days ago · ... 넷플릭스 '흑백요리사: 요리 계급 전쟁'(이하 '흑백요리사')의 최종 우승자가 '나폴리 맛피아' 권성준(30)씨로 확정된 가운데 권씨가 우승 소감을 밝혔다."}, {'url': 'https://www.mk.co.kr/news/society/11138329', 'content': "3 days ago · 넷플릭스 '흑백요리사' 최종 우승자인 나폴리 맛피아(권성준)가 방송 후 테이블 수를 줄인 이유를 밝혔다. 11일 유튜브 채널 '백종원'에는 '손님이\xa0..."}, {'url': 'https://www.youtube.com/watch?v=1OKUP6NhGAE', 'content': "6 days ago · 있습니다. 숱한 화제를 남긴 넷플릭스 '흑백요리사: 요리 계급 전쟁' 마지막화가 드디어 공개됐습니다. 최종우승자는 ...Duration: 1:25Posted: 6 days ago"}, {'url': 'https://sports.khan.co.kr/article/202410090945003', 'content': "6 days ago · 8일 오

In [28]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 SNS 게시글 형태로 100자 내외로 작성하세요."},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
넷플릭스 '흑백요리사'의 최종 우승자는 '나폴리 맛피아' 권성준! 그의 열정과 실력이 빛난 순간이었습니다. 축하합니다! #흑백요리사 #권성준 #우승


In [29]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변에 우승자의 소감도 찾아줘."},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 넷플릭스 흑백요리사 권성준 우승 소감
Log: 
Invoking: `tavily_search_results_json` with `{'query': '넷플릭스 흑백요리사 권성준 우승 소감'}`



[관찰 내용]
Observation: [{'url': 'https://v.daum.net/v/20241009143204433', 'content': '[동아닷컴] 넷플릭스 \'흑백요리사\' 우승자 나폴리 마피아(권성준)가 소감을 전했다. 지난 8일 권성준은 자신의 SNS에 \'흑백요리사\' 우승 소감을 밝혔다. 그는 "기쁨의 소감보다는 먼저 사과와 감사의 말을 올리고 싶다"면서 "7개월이라는 시간 동안 우승 소식을 어렵게 숨기다 막판에 저도 ...'}, {'url': 'https://sports.donga.com/article/all/20241009/130182223/1', 'content': "넷플릭스 '흑백요리사' 우승자 나폴리 마피아(권성준)가 소감을 전했다. 지난 8일 권성준은 자신의 SNS에 '흑백요리사' 우승 소감을 밝혔다 ..."}, {'url': 'https://news.jtbc.co.kr/article/article.aspx?news_id=NB12218085', 'content': "넷플릭스 예능 '흑백요리사: 요리 계급 전쟁'(이하 '흑백요리사')의 나폴리 맛피아(권성준)가 우승 소감을 지난 8일 전했다. 나폴리 맛피아는 자신의 sns를 통해 '기쁨의 소감보다는 먼저 사과와 감사의 말을 올리고 싶다'고 밝혔다."}, {'url': 'https://www.joongang.co.kr/article/25283019', 'content': '넷플릭스 \'흑백요리사: 요리 계급 전쟁\' (이하 \'흑백요리사\') 최종 우승자 \'나폴리 맛피아\' 권성준 (30)씨가 세미파이널에서 맞붙은 셰프들을 향해 "내가 건방지게 굴었다"며 사과했다. 권씨는 지난 8일 인스타그램을 통해 

#### [실습] PDF를 활용한 RAG + 실시간 웹 검색을 활용한 챗봇 구축
 - 자유주제 가능
  
 - (예시) 요리 레시피 검색 및 추천 에이전트  
    목표: PDF로 제공된 요리책에서 원하는 레시피를 검색하고, 찾을 수 없으면 웹에서 정보를 검색해 추천 요리법을 제공하는 시스템을 구축
    
 - (예시) 여행 정보 검색 및 추천 에이전트  
   목표: PDF 문서로 제공된 여행 가이드북에서 특정 목적지에 대한 정보를 검색하고, 부족한 정보는 웹에서 검색해 종합적인 여행 계획을 제시하는 시스템을 구축

 - (예시) 논문 검색 및 관련 연구 서베이 에이전트
   목표: 공모전 혹은 프로젝트에 사용할 딥러닝 모델을 논문 RAG 문서로 활용하여, 논문 질의 응답을 받은 후 부족한 정보다 비교 모델을 웹에서 검색해 사용할 모델들을 정리해주는 시스템을 구축

In [31]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력0
logging.langsmith("Agents")

import os

os.environ['LANGCHAIN_PROJUECT'] = 'Agents'

from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성
# k=6은 검색 결과를 6개까지 가져오겠다는 의미
search = TavilySearchResults(k=6)

# 검색 결과
search.invoke("가짜 뉴스란 무엇인가요?")



LangSmith 추적을 시작합니다.
[프로젝트명]
Agents


[{'url': 'https://ko.wikipedia.org/wiki/%EA%B0%80%EC%A7%9C%EB%89%B4%EC%8A%A4',
  'content': '가짜 뉴스는 뉴스로 제시된 허위 또는 오해의 소지가 있는 정보이다. 가짜 뉴스는 종종 개인이나 단체의 평판을 손상시키거나 광고수익을 통해 돈을 벌려는 목적을\xa0...'},
 {'url': 'https://www.voakorea.com/a/world_us-election-abc_fake-news/6033936.html',
  'content': 'Jul 25, 2020 · 가짜 뉴스는 다시 정의하면 SNS나 전통적인 뉴스 기사 등을 통해 금전적 혹은 정치적 이득을 얻기 위해 오인할 의도를 가지고 유포하는 잘못된 정보라고\xa0...'},
 {'url': 'https://www.miline.or.kr/board/view?pageNum=1&rowCnt=12&no1=4&linkId=889&menuId=MENU00364&schType=0&schText=&boardStyle=Text&categoryId=&continent=&country=&consonant=&listKeyWordGubun=null&crawlingSiteType=null',
  'content': "한국언론진흥재단에서는 가짜뉴스를 '정치적 경제적 이익을 목적으로 한 의도적인 언론보도 형 식으로 유포한 거짓 정보'로 정의한다. 악성루머, 소문, 유언비어와 같은\xa0..."},
 {'url': 'https://www.khan.co.kr/opinion/column/article/202109020300105',
  'content': 'Sep 2, 2021 · 지금까지 논의된 바에 따르면 국내에서도 가짜뉴스는 ①정치·경제적 이익을 목적으로 ②고의로 왜곡·날조하고 ③언론 보도로 가장하는 거짓 정보로 대략 정의\xa0...'},
 {'url': 'https://www.haeundae.go.kr/board/view.do?boardId=BBS_0000

# "가짜뉴스란 무엇인가" PDF 논문을 활용한 Retriever 문서 조회

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("가짜뉴스란 무엇인가.pdf")

# 텍스트 분할기를 사용하여 문서를 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성
retriever = vector.as_retriever()

In [34]:
# 문서에서 관련성 높은 문서를 가져옴
retriever.invoke("가짜뉴스의 정의를 알려줘")

[Document(metadata={'source': '가짜뉴스란 무엇인가.pdf', 'page': 2}, page_content='가짜뉴스(fake news)란 무엇인가?   175\n우리가 이 기사에서 혼란을 느끼는 것은 “가짜뉴스란 무엇인가 ?”, “어\n디까지를 가짜뉴스로 보아야 하는가 ?”에 대한 명확한 기준을 갖고 있지 \n못하기 때문이다 . 최근 학자 , 언론현장 , 정치권 등을 중심으로 벌어지고 \n있는 가짜뉴스 논쟁도 이에 대한 개념과 범위의 모호성 때문이다 . \n가짜뉴스를 둘러싼 백가쟁명 (百家爭鳴 )식 논란이 계속되고 있지만 , 이\n에 대한 용칭 , 정의, 그리고 범위를 어떻게 봐야 하는지에 대한 명확한 \n기준은 아직 없다 (박아란 , 2017; 염정윤ㆍ정세훈 , 2018). 누구나 동의할 \n정도의 합의된 개념이 있는 것도 아니다 . 오히려 반대로 가짜뉴스에 대\n한 정의가 너무 다르고 , 많아서 혼란스럽기까지 하다 . 그렇다고 중구난\n방(衆口難防 ) 식으로 흘러 다니는 가짜뉴스의 개념을 정돈 없이 마구 쓸 \n일도 아니다 . 가짜뉴스를 둘러싸고 벌이는 사회적 , 정치적 논쟁에 따른 \n소모가 너무 크기 때문이다 . 사람마다 가짜뉴스를 코에 걸면 코걸이 , 귀\n에 걸면 귀걸이 식으로 제멋대로 해석하는 한 사회적 갈등은 줄어들지 \n않는다 . 사회적 갈등을 줄이기 위해서라도 산만하게 흩어져 있는 가짜뉴\n스에 대한 개념과 범위를 정리하고 넘어갈 필요가 있다 . \n뉴스의 진위 (眞僞)를 가리기 위해서는 누구나 동의할 수 있는 원칙이\n나 기준이 필요하다 . 그러나 지금까지 이뤄진 가짜뉴스 연구의 대부분은 \n가짜뉴스에 대한 규제문제에 치중해 왔다 . 가짜뉴스의 개념을 부분적으\n로 설명한 연구가 전혀 없는 것은 아니지만 (이인철 , 2018; 유의선 , 2018; \n윤성옥 , 2018; 황용석ㆍ권오성 , 2017; Tandoc Jr., Lim & Ling, 2018), \n이를 체계적으로 검토하고  논의한 연구는 없다 . \n이런

In [35]:
from langchain.tools.retriever import create_retriever_tool


retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야함!!
)

In [36]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

In [37]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [38]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

In [39]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [40]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

In [41]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "기술적 관점의 가짜뉴스에 대해 알려주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 기술적 관점의 가짜뉴스
Log: 
Invoking: `tavily_search_results_json` with `{'query': '기술적 관점의 가짜뉴스'}`



[관찰 내용]
Observation: [{'url': 'https://www.hani.co.kr/arti/economy/it/868771.html', 'content': 'Nov 4, 2018 · 가짜 뉴스를 탐지하는 기술적 접근은 주로 메시지의 언어적 신호와 네트워크 분석을 통해 이뤄지고 있다. 가짜 뉴스에 자주 등장하는 단어와 표현을 인공\xa0...'}, {'url': 'http://m.dongascience.com/news.php?idx=17999', 'content': 'May 12, 2017 · 가짜 뉴스를 자동으로 찾는 기술에는 문장을 인식해 신뢰할 만한 데이터와 일일이 대조하는 방법과, 루머의 전파, 언어 특성을 토대로 선별하는 방법 등이\xa0...'}, {'url': 'https://www.lawtimes.co.kr/LawFirm-NewsLetter/195918', 'content': 'Feb 14, 2024 · 딥페이크 기술을 이용해 유명인사의 얼굴을 합성한 가짜 뉴스가 인터넷에서 난무하고 있으며, 이로 인해 정치적이나 사회적 불안감이 가중되고 있는 상황\xa0...'}, {'url': 'https://modulabs.co.kr/blog/fake-news-and-fact-check/', 'content': 'Mar 4, 2024 · 가짜 뉴스 탐지는 사실성과 합성성에 기반하여 이루어지게 됩니다. 자세히 이야기하면, AI가 작성한 뉴스도 사실이라면 탐지에서 제외를 시킬 지, 혹은\xa0...'}, {'url': 'https://post.naver.com/viewer/postView.naver?volumeNo=31260905&memberNo=253010&searchKeyword=%EA%B0%80

In [42]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "가짜뉴스에 대한 다차원적 논의에 문서에서 찾아서 알려줘."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 가짜뉴스 다차원적 논의
Log: 
Invoking: `pdf_search` with `{'query': '가짜뉴스 다차원적 논의'}`



[관찰 내용]
Observation: 이런 차원에서 이 글은 가짜뉴스를 어떻게 개념화하고 , 어느 범위까지
로 볼 것인가에 대한 소고 (小考)이다. 이를 위해 이 글은 기존 연구자들
이 가짜뉴스를 어떤 관점에서 정의 , 설명하고 있는지 , 그 한계는 무엇인
지, 그리고 타당성 있는 개념은 무엇인지를 체계적으로 살펴보고자 한
다. 이를 위해 다음의 네 가지 차원 , 즉 법률적 관점 , 저널리즘적 관점 , 
기술적 관점 그리고 정치사회적 관점에서 가짜뉴스의 개념과 범위에 대한 의미를 논의해 보고자 한다 . 법률적 관점은 가짜뉴스를 법리적 측면

184  미디어와 인격권 ▪제4권 ▪제2호 ▪2018
념이나 범위와 같은 미시적 차원 , 그리고 가짜뉴스가 생성 , 유포되는 사
회 맥락에 대한 거시적 차원을 통합적으로 제시해 줄 것으로 기대해 볼 
수 있다 . 
1. 법률적 관점에서의 가짜뉴스 
문제의 가짜뉴스를 사회적 통제나 규제의 대상으로 볼 수 있는지 판
단하기 위해서는 먼저 이것이 법률적으로 어떻게 규정되는가에 대한 검토가 앞서야 한다 . 가짜뉴스에 대한 법률적 개념은 가짜뉴스를 제재의 
대상으로 보고 정부나 사법기관이 개입해도 무방한 대상인지를 결정하
는 기준이 된다 . 이를 위해서는 전문가의 의견과 함께 법률이나 주요 법
안들이 규정하는 가짜뉴스의 의미를 되짚어 볼 필요가 있다 . 클라인과 
우엘러 (Klein & Wueller, 2017) 는 ‘가짜뉴스에 대한 법률적 관점 (fake 
news: a legal perspective)’ 이라는 논문에서 가짜뉴스의 법적 기준으로 
의도된 허위성 (falsity)과 뉴스의 형식성 (formality) 을 주요 요건으로 꼽았
다. 이들은 실체적 진실 여부와 뉴스의 보편적인 양식을 갖췄는지를 기
준으로 가짜뉴스를 판단할 것을

In [43]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [44]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "가짜뉴스에 대한 다차원적 논의에 문서에서 찾아서 알려줘."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 가짜뉴스 다차원적 논의
Log: 
Invoking: `pdf_search` with `{'query': '가짜뉴스 다차원적 논의'}`



[관찰 내용]
Observation: 이런 차원에서 이 글은 가짜뉴스를 어떻게 개념화하고 , 어느 범위까지
로 볼 것인가에 대한 소고 (小考)이다. 이를 위해 이 글은 기존 연구자들
이 가짜뉴스를 어떤 관점에서 정의 , 설명하고 있는지 , 그 한계는 무엇인
지, 그리고 타당성 있는 개념은 무엇인지를 체계적으로 살펴보고자 한
다. 이를 위해 다음의 네 가지 차원 , 즉 법률적 관점 , 저널리즘적 관점 , 
기술적 관점 그리고 정치사회적 관점에서 가짜뉴스의 개념과 범위에 대한 의미를 논의해 보고자 한다 . 법률적 관점은 가짜뉴스를 법리적 측면

184  미디어와 인격권 ▪제4권 ▪제2호 ▪2018
념이나 범위와 같은 미시적 차원 , 그리고 가짜뉴스가 생성 , 유포되는 사
회 맥락에 대한 거시적 차원을 통합적으로 제시해 줄 것으로 기대해 볼 
수 있다 . 
1. 법률적 관점에서의 가짜뉴스 
문제의 가짜뉴스를 사회적 통제나 규제의 대상으로 볼 수 있는지 판
단하기 위해서는 먼저 이것이 법률적으로 어떻게 규정되는가에 대한 검토가 앞서야 한다 . 가짜뉴스에 대한 법률적 개념은 가짜뉴스를 제재의 
대상으로 보고 정부나 사법기관이 개입해도 무방한 대상인지를 결정하
는 기준이 된다 . 이를 위해서는 전문가의 의견과 함께 법률이나 주요 법
안들이 규정하는 가짜뉴스의 의미를 되짚어 볼 필요가 있다 . 클라인과 
우엘러 (Klein & Wueller, 2017) 는 ‘가짜뉴스에 대한 법률적 관점 (fake 
news: a legal perspective)’ 이라는 논문에서 가짜뉴스의 법적 기준으로 
의도된 허위성 (falsity)과 뉴스의 형식성 (formality) 을 주요 요건으로 꼽았
다. 이들은 실체적 진실 여부와 뉴스의 보편적인 양식을 갖췄는지를 기
준으로 가짜뉴스를 판단할 것을

In [45]:
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
The document discusses the multidimensional discourse on fake news from the following four perspectives:

1. **Legal Perspective**: It examines how fake news can be defined and regulated legally. The legal criteria for fake news include intended falsity and the formality of the news. Additionally, there is a discussion on whether fake news can be viewed as a subject of social control.

2. **Journalistic Perspective**: The discussion revolves around how to understand and respond to fake news in light of journalistic standards and ethics. This includes how fake news undermines the credibility of journalism.

3. **Technological Perspective**: It analyzes how fake news is generated and disseminated in the digital media environment. The impact of technological factors on the spread of fake news is examined, including the role of social media and algorithms.

4. **Political-Social Perspective**: The exploration focuses on how fake news functions as a phenomenon in political communica

# Agent 템플릿

In [48]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser

########## 1. 도구를 정의 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성
# k=6은 검색 결과를 6개까지 가져오겠다는 의미
search = TavilySearchResults(k=6)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("./가짜뉴스란 무엇인가.pdf")

# 텍스트 분할기를 사용하여 문서를 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성
retriever = vector.as_retriever()


retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### 1-3. tools 리스트에 도구 목록을 추가 ###
# tools 리스트에 search와 retriever_tool을 추가
tools = [search, retriever_tool]

########## 2. LLM 을 정의  ##########
# LLM 모델을 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 를 정의##########

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의 ##########

# 에이전트를 생성
# llm, tools, prompt를 인자로 사용
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리를 추가 ##########

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의합니다. ##########
agent_stream_parser = AgentStreamParser()

In [49]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input":  "문서의 소결론에 대해 찾아서 요약해줘."},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 소결론
Log: 
Invoking: `pdf_search` with `{'query': '소결론'}`



[관찰 내용]
Observation: 해 최종 판단을 유보한 채 중립적인 태도로 보도했다면 그것은 가짜뉴
스로 볼 수 없다고 말한다. 언론이 보도를 통해 제기한 의혹 자체를 가
짜뉴스로 볼 수는 없다는 뜻이다. 
문제는 무엇이 진짜인지, 가짜인지를 구분하는 기준을 정하기 어렵기 
때문이다(오세욱 등, 2017). 세상에 절대적 진실은 존재하지 않는다. 언
론이 작성, 보도하는 기사는 진실에 비교적 가까운 ‘종합적 진실(the

이다. 기사 작성주체가 반드시 전통적인 기존의 언론사나 기자일 필요도 
없다. 
가짜뉴스의 또 다른 논쟁점 가운데 하나가 규제의 문제이다. 가짜뉴스
를 제도적으로 규제하는 것이 온당한가, 현실적으로 가능한 일인가, 그
리고 무엇을 어떻게 규제할 것인가에 대한 사회적 합의나 기준이 없다.

에서 사실성 검정은 게이트키핑(gate keeping) 과정을 거쳤는지가 중요하
다. 모두 그러하지는 않지만 이름이 없는 매체일수록 게이트키핑 구조를 
갖추고 있지 못할 개연성이 크기 때문이다. 게이트키핑 과정이 없을수록 
거짓 또는 허위정보를 생산할 가능성이 더 높다. 정보의 신뢰성은 정보

현의 자유권을 훼손해서는 안 된다는 입장이다. 하지만 가짜뉴스를 무제
한 허용해야 한다는 의견이 전적으로 지지되는 것도 아니다. 정치권이나 
정부 일각에서 가짜뉴스의 무분별한 유통에 대해 제재가 필요하다고 본
다. 예를 들어 폭력적이거나 선정적인 동영상물에 대한 유통을 차단하고
[최종 답변]
문서의 소결론에서는 가짜뉴스의 정의와 규제에 대한 논쟁을 다루고 있습니다. 가짜뉴스를 구분하는 기준을 정하기 어렵고, 절대적 진실이 존재하지 않기 때문에 언론이 제기한 의혹 자체를 가짜뉴스로 볼 수 없다는 점을 강조합니다. 또한, 가짜뉴스를 제도적으로 규제하는 것에 대한 사회적 합의나 기준이 없으며, 정보의 신뢰성

In [53]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {
        "input": "가짜뉴스를 걸러낼 수 있는 키워드들을 찾아서 요약적으로 서술해주세요. 한글로 답변하세요"
    },
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 가짜뉴스 걸러내는 키워드
Log: 
Invoking: `tavily_search_results_json` with `{'query': '가짜뉴스 걸러내는 키워드'}`



[관찰 내용]
Observation: [{'url': 'https://brunch.co.kr/@itrendlab/209', 'content': "가짜 뉴스를 걸러내는 읽기 방법을 《핵심 읽기 최소원칙》에 쓴 내용을 중심으로 정리했습니다. 가짜 뉴스, 가짜 정보가 사회 문제로 보도된다. 우리가 '가짜 뉴스'라고 말하는 정보 중에는 거짓이 교묘하게 숨어있다. ... 글을 읽기보다 키워드 검색 결과에 ..."}, {'url': 'https://www.mk.co.kr/news/columnists/10861942', 'content': '한 개의 가짜뉴스에 대한 팩트체크를 하는 동안 열 개 이상의 가짜뉴스가 만들어지고 있고, 딥페이크 등 기술 발달로 가짜뉴스 가려내기는 점점 힘들어지고 있다. 가짜뉴스를 수익형 사업모델로 삼는 1인 미디어도 갈수록 늘고 있다. 가짜뉴스를 걸러내고 비판적 ...'}, {'url': 'https://www.hankookilbo.com/News/Read/201612122085264551', 'content': "가짜뉴스 걸러내는 세 가지 방법, URLㆍ소개글ㆍ취재원. 가짜뉴스는 독자들의 '무신경'을 먹고 자란다고 해도 과언이 아니다. 구글과 페이스북이 ..."}, {'url': 'https://www.seoul.co.kr/news/newsView.php?id=20230914010001', 'content': '가짜뉴스 걸러내는 메커니즘 만들어야" 입력 : 2023-09-14 01:26 ㅣ 수정 : 2023-09-15 08:36 \'가짜뉴스, 어떻게 근절할 것인가\' 서울신문 주최 전문가 4인 ...'}, {'url': 'https://www.ngonews.kr/news/articleView

In [52]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 SNS 게시글 형태로 500자 내외로 작성하세요."},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
가짜뉴스를 딥러닝과 머신러닝 기술로 효과적으로 걸러낼 수 있는 방법을 소개합니다! 먼저, 뉴스 데이터를 수집하고 전처리하는 과정이 필요합니다. 이 단계에서는 텍스트 정규화, 불용어 제거, 형태소 분석 등을 통해 데이터의 품질을 높입니다. 그런 다음, 전처리된 데이터를 활용하여 딥러닝 모델을 학습시킵니다. CNN, RNN, LSTM 등 다양한 딥러닝 아키텍처를 사용할 수 있으며, 앙상블 기법을 통해 여러 모델의 예측 결과를 결합하여 성능을 향상시킬 수 있습니다.

모델 학습 후에는 머신러닝 기법을 통해 뉴스의 특징을 추출하고, 이를 기반으로 가짜뉴스 여부를 분류합니다. 이 과정에서는 TF-IDF, 워드 임베딩 등의 기법을 활용하여 텍스트의 의미를 파악합니다. 마지막으로, 모델의 성능을 평가하고 하이퍼파라미터 튜닝이나 데이터셋 확장을 통해 모델을 개선합니다. 이러한 과정을 통해 가짜뉴스 탐지의 정확도를 높일 수 있습니다.

이러한 기술들은 가짜뉴스의 확산을 방지하고, 사용자에게 신뢰할 수 있는 정보를 제공하는 데 큰 기여를 할 수 있습니다. 딥러닝과 머신러닝을 활용한 가짜뉴스 탐지, 이제는 필수입니다! #가짜뉴스 #딥러닝 #머신러닝 #데이터과학 #인공지능 #AI #뉴스검증
